<a href="https://colab.research.google.com/github/Abanoub1111/GBG_TASKS/blob/main/Session-2/GBG_s2_(Prompt_strategies%2C_Sentiment_analysis_using_LLM_with_2_approaches_Reliability_%26_Reasoning).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# testing dataset

In [2]:
import pandas as pd
df = pd.read_parquet("hf://datasets/ehab215/egyptian_sentiment_analysis_dataset/data/train-00000-of-00001.parquet")

In [3]:
df

,text,sentiment,__index_level_0__
0,بصراحه الموبايل الجديد ده رائع جداً، البطاريه ...,positive,0
1,أنا مش فاهم إزاي حد يشتري من المحل ده، الأسعار...,negative,1
2,شفت المباراه النهارده وكانت حاجه تجنن بجد، الل...,positive,2
3,المسلسل الجديد ده زفت أوي، الحبكه ممله والممثل...,negative,3
4,بجد النادي الرياضي ده فخم جداً، المرافق حديثه ...,positive,4
...,...,...,...
5815,الجو النهارده كان حلو أوي، بس للأسف مكنتش قادر...,neutral,5817
5816,كل ما بفكر أغير شغلي بلاقي نفسي متردده، عشان ب...,neutral,5818
5817,الحياه في القاهره بقت غاليه أوي، المصاريف عالي...,neutral,5819
5818,عايز أبدأ أمارس رياضه جديده، بس مش عارف أختار ...,neutral,5820


In [4]:
df = df.drop('__index_level_0__', axis=1)

# LLM

In [12]:
!pip install langchain_groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 kB 8.5 MB/s eta 0:00:00


In [13]:
import os
from google.colab import userdata
from langchain_groq import ChatGroq
import json

In [17]:
api_key = userdata.get("groq-api")

llm = ChatGroq(
    model="meta-llama/llama-4-maverick-17b-128e-instruct",
    api_key=api_key,
    temperature = 0
    )

# Reasoning approach

In [20]:
def sentiment_reasoning(text):
    prompt = f"""
You are a sentiment analysis model.

Task:
Classify the sentiment of the given text as one of:
- positive
- negative
- neutral

Then provide:
- sentiment label
- confidence score between 0 and 1
- a short rationale (1–2 sentences, no step-by-step reasoning)
- key phrases from the text that influenced the decision

Text:
"{text}"

Return the result strictly as JSON.
"""

    response = llm.invoke(prompt)

    # Gemini may return markdown-wrapped JSON, so we clean it
    content = response.content.strip()

    if content.startswith("```"):
        content = content.replace("```json", "").replace("```", "").strip()

    model_output = json.loads(content)

    result = {
        "task": "sentiment_analysis",
        "approach": "reasoning",
        "input": {"text": text},
        "output": model_output
    }

    return result

In [21]:
answer = sentiment_reasoning(df['text'][0])
print(answer)

{'task': 'sentiment_analysis', 'approach': 'reasoning', 'input': {'text': 'بصراحه الموبايل الجديد ده رائع جداً، البطاريه بتفضل شحنه كتير والكاميرا بتاخد صور تحفه'}, 'output': {'sentiment': 'positive', 'confidence': 0.95, 'rationale': 'The text expresses a very positive opinion about the new mobile phone, highlighting its long-lasting battery and excellent camera.', 'key_phrases': ['رائع جداً', 'تحفه']}}


In [24]:
results = []

for i in range(10):
    text = df['text'][i]
    output = sentiment_reasoning(text)
    results.append(output)

with open("sentiment_results_reasoning.json", "w", encoding="utf-8") as f:
    json.dump(results, f, indent=2, ensure_ascii=False)

# Reliability approach

In [25]:
import json

def sentiment_reliability(text):
    prompt = f"""
You are a sentiment classification model.

Task:
Classify the sentiment of the given text as one of:
- positive
- negative
- neutral

Return ONLY:
- sentiment label
- confidence score between 0 and 1

Do NOT provide reasoning.
Do NOT explain the decision.
Return strictly valid JSON.

Text:
"{text}"
"""

    response = llm.invoke(prompt)

    content = response.content.strip()

    # Clean markdown-wrapped JSON if present
    if content.startswith("```"):
        content = content.replace("```json", "").replace("```", "").strip()

    model_output = json.loads(content)

    result = {
        "task": "sentiment_analysis",
        "approach": "reliability",
        "input": {"text": text},
        "output": model_output
    }

    return result

In [26]:
results = []

for text in df['text'].iloc[:10]:
    output = sentiment_reliability(text)
    results.append(output)
with open("sentiment_results_reliability.json", "w", encoding="utf-8") as f:
    json.dump(results, f, indent=2, ensure_ascii=False)